In [ ]:
# Clean, Transform, Merge, Reshape

In [3]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

## 데이터 합치기

#### 1. merge
- 수평(횡) 결합
- merge의 how 옵션에 따라 inner, outer, left, right join을 발생시킴
- key가 다르지만 같이 넣고 싶다면 left on = key1, right on = key2
- inner join : 교집합, full join : 합집합, left join : left + inner, right join : right + inner
- key와 value 중에 기준을 하고 싶다? on = 'key' / 여러 개의 key를 list로 줘도 됨
- suffix : left = x, right = y 이므로 선택하고 싶은 value에따라 고를 것.

In [3]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [5]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [9]:
pd.merge(df1, df2) # 살아남은 교집합만 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [27]:
# left on, right on
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
print(df3)
print(df4)
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [11]:
# how
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

pd.merge(df1, df2, on='key', how='left')
# 같은 값의 key값이 있다면 catesian production된다. left key : 2, right key : 2라면 2 x 2개의 key가 생성.
# 고로 key 'a'가 4개임.
# inner에서만 발생한다. why? inner에서만 중복처리할 일이 있기 때문이다.

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [14]:
# 두 개의 key
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
print(left)
print(right)
pd.merge(left, right, on=['key1', 'key2'], how='outer')

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [15]:
# suffix
pd.merge(left, right, on='key1') # suffix = default

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [12]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) 

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
# left : on = key, right : index = True
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [20]:
# 계층적 색인 (multi-index)
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [21]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [19]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [22]:
# 두 개의 계층적 색인 (multi index)
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [23]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### 2. join
- catesian production 시 에러 why? index를 기준으로 합치기 때문이다. 이를 좀만 생각하면 왜 join은 sort = False인지 알 수 있음
- index로 merge할 때, df.join 메소드 사용
- col이 겹치지 않으며 색인구조가 유사한 여러 df 병합 시 사용
- 데이터를 얼마나 살리고 죽이느냐에 따라 join의 how를 결정함. ex) 결측치 많아도 합친다 = outer
- left.join(other)  // how의 default는 left 이므로 df : 중요한 data, other : 합쳐올 data

#### 연산의 과정을 생각하면 join이 저렴하고 효율적, merge는 비싸고 오래 걸리는 연산이다. why? catesian production

In [30]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [31]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [36]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [32]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [39]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [41]:
left2 # left2가 기준이므로 얘가 가진 index를 기준으로 생각할 것.

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [40]:
# 예를 들어 3개의 부서가 각각 2개씩 도시를 모아오면 합쳐주는 것.
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [42]:
# 그냥 다 합쳐
left2.join([right2, another], how='outer') 

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [38]:
# look up table 
left1.join(right1, on='key') # right : look up table, 유일한 값들이 표 형식으로 존재하여 다른 객체들이 참조함.

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


___

#### concatenating along an axis
- 수직(종) 결합(concatenating)
- 이어 붙이기(binding)
- 적층(stacking)
- 동일한 열이름을 갖는 df들의 수직연결(행연결)을 위해 사용
- pandas.concat(objs, ignore_index, keys, levels, names, verify_integrity) # 결합할 축에 관련된 옵션
- pandas.concat(join, axis, join_axes, copy) # 결합할 형식에 대한 옵션
- keys
    - 어느 부서가 조사했는지 행결합 시 표시하기.
    - multi indexing이 필요하다. how? keys = '어느 부서', names = '고객 ID'
- verify_index
    - 중복된 행들이 발생하면 error를 발생시킴 / 뭐가 더 최신 내용인지 확인할 수 있도록 해줌.
- ignore index
    - index 무시하고 합침.
- join
    - default : 'outer' 
    - col은 전부 다 합쳐주지만 없는 df에서는 전부 NaN이 나올 것.
    - why? 고객의 segment를 나누고 조사하는 항목이 다를 수 있으므로

#### pd.concat의 작동 원리
1. 결합축(axis=0)으로 이어붙인다.
2. 결합축이 아닌(axis=1) index로 join이 이뤄진다.
   - 동일한 index label(col_name)을 key로 하는 join이 생성
   - 암묵적으로 join=outer이므로, NaN이 발생할 수 있다.
   - 특정 df 내에서 join key가 중복된 경우 error 발생. (행 결합 시 row_name이 중복되면 error 난다.)
   - 즉, cartesian production을 피한다.
3. igore_index = True인 경우:
   - 입력 df의 결합축 index를 무시하고, 
   - 출력 df에서는 순서 index가 생성된다.
4. keys= [ex) 부서이름]가 제공된 경우:
   - 출력 df에서 결합축 최외곽의 계층적 색인 label이 지정된다.
   - 이때, names=[]가 제시된 경우 계층적 색인들의 level이 지정된다.
5. verify_integrity = True인 경우: 
   - 출력 df에서 index의 중복여부를 검사하여
   - 중복된 경우, ValueError: Indexes have overlapping values: ['a', 'b']


In [43]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [44]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [45]:
np.concatenate([arr, arr], axis = 1) # axis에 따라 join, merge처럼 수평 결합도 지원함.

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [49]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])

print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [50]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [51]:
pd.concat([s1, s2, s3], axis = 1) # outer join

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [53]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [56]:
# Series 2개 열방향 결합
pd.concat([s1, s4])

a    0
b    1
a    0
b    5
f    5
g    6
dtype: int64

In [62]:
# multi indexing 
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three']) # keys = '부서 이름'
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

### 이 밑으로는 몰라도 됨. 그냥 join이랑 merge 쓰셈
___

In [55]:
# Series 2개 수평 결합
pd.concat([s1, s4], axis=1)

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [61]:
# a, c, b, e에 대해 수평 결합
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [75]:
# 2개의 df 수평 결합
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
print(df1)
print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [78]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2']
         , names=['upper', 'lower']) # 이름 지정해주기

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [77]:
# dict로 넣어도 똑같은 결과
pd.concat({'level1': df1, 'level2': df2}, axis=1)

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [94]:
# 2개의 df 수직 결합
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

df1

,a,b,c,d
0,-1.265934,0.119827,-1.063512,0.332883
1,-2.359419,-0.199543,-1.541996,-0.970736
2,-1.307030,0.286350,0.377984,-0.753887


In [95]:
df2

,b,d,a
0,0.331286,1.349742,0.069877
1,0.246674,-0.011862,1.004812


In [82]:
pd.concat([df1, df2], ignore_index=True) # index 이름 무시하고 새로 재편성

C:\Users\tmznq\Anaconda3\envs\ml_scratch\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


In [96]:
pd.concat([df1, df2], axis = 1) # 열이 중복되면 에러가 나야하는데 왜 되나요?

,a,b,c,d,b,d,a
0,-1.265934,0.119827,-1.063512,0.332883,0.331286,1.349742,0.069877
1,-2.359419,-0.199543,-1.541996,-0.970736,0.246674,-0.011862,1.004812
2,-1.307030,0.286350,0.377984,-0.753887,NaN,NaN,NaN


___

- 헷갈리지 않으려면 수직은 concat, 수평은 merge, join만 사용할 것. 

### 수평과 수직 구분하는 예시
- concat : 고객들을 여러 개로 나눠서 조사함. 조사하는 항목은 전부 같음
- merge, join : 고객에게 여러 가지 항목을 조사함. 고객은 전부 같지만 조사할 내용이 다름.

___

#### combining and data overlab
- 결측값을 갖고 있는 경우, 다른 성분으로 대체하기

In [1]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

NameError: name 'Series' is not defined

In [103]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [104]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

- np.where(condition, b, a):
    - condition[i]= True면 k[i] = b[i]
    - condition[i]= False면 k[i] = a[i]

In [105]:
np.where(pd.isnull(a), b, a) # np.array 비교니까 index는 무시함. why? Series = index + array

array([0. , 2.5, 2. , 3.5, 4.5, nan])

- pandas 객체 메소드 combine_first
- s = s1.combine_first(s2):
    - s1['label'] = NaN이면 s['label'] = s2['label']
    - s1['label'] = NaN이 아니면 s['label'] = s1['label']
    - np.where와 달리 index의 label로 비교한다는 차이가 있다.

In [109]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [110]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [108]:
S = b[:-2].combine_first(a[2:]) # 합집합인데 s1이 NaN일 때 s2로 대체, 아니면 s1의 값 그대로 적용
S

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [112]:
# df에서는 어떻게 적용되는가?
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
print(df1)
print(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [111]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


___
## stack, unstack
- index와 column을 row 사이에 왔다갔다하게 만드려고 사용함.
- index -> col : unstack
- col -> index : stack

In [63]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [64]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [65]:
result.unstack(0) # level을 고르는 것이므로 level = 0이면 최외각

,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [66]:
# 예시
df = DataFrame(np.arange(16).reshape((4,4)), index = [['one', 'one', 'two', 'two'], [1, 2, 3, 4]], 
               columns=[['k','l','m', 'n'], ['a', 'a', 'b',' b']])
df

k   l   m   n
        a   a   b   b
one 1   0   1   2   3
    2   4   5   6   7
two 3   8   9  10  11
    4  12  13  14  15

In [74]:
df.unstack()

k                    l                    m                     n       \
       a                    a                    b                     b        
       1    2    3     4    1    2    3     4    1    2     3     4    1    2   
one  0.0  4.0  NaN   NaN  1.0  5.0  NaN   NaN  2.0  6.0   NaN   NaN  3.0  7.0   
two  NaN  NaN  8.0  12.0  NaN  NaN  9.0  13.0  NaN  NaN  10.0  14.0  NaN  NaN   

                 
                 
        3     4  
one   NaN   NaN  
two  11.0  15.0

#### Reshaping with hierarchical indexing
- stack, unstack과 동일하며 적용 시 발생하는 NaN을 없애는 기능도 있음.
- df는 reshape가 안된다. why? dtype이 다름
___

In [5]:
# stack예시 1
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [7]:
result.shape

(6,)

In [12]:
result.unstack(0).stack()

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int32

In [27]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [15]:
# stack 예시 2
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [23]:
data2.unstack().stack(dropna=False) # stack의 dropna 옵션 사용

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

___
## 데이터 pivot
- DB나 csv 파일 등에 기록된 시계열 데이터는 일반적으로 시간순으로 기록된다.
     - 이때 시간순으로 각 항목을 모두 레코드로 나열한 방식을 long data라 한다. 
     - row index : datetime이 찍힘
     - long data 형식이 분석에 적절하지 않을때, 각 항목을 열로 변경한 것을 wide data라 한다.

- wide data 를 long data
    - df.stack()
    - df.reset_index()

- long data를 wide data
    - df.pivot
    - 이때 지정한 index-columns 값의 조합은 unique 해야 한다.

In [ ]:
# pivot 예시1

In [44]:
data = pd.read_csv('C:/Users/tmznq/workspace/ml_scratch/seoul_coding_academy/pydata-book-1st-edition/ch07/macrodata.csv')

In [45]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [37]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods # 연도와 1개의 분기 주기를 가지고 있는 period

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [46]:
data.to_records()[:3] # 1개의 row를 각각의 record로 만듦.
# But 열 기준 데이터인데 이를 행 기준으로 만든 데이터이므로 기존의 데이터를 copy해옴. 
# 이는 데이터를 비 효율적으로 쓰는 것이다.

rec.array([(0, 1959., 1., 2710.349, 1707.4, 286.898, 470.045, 1886.9, 28.98, 139.7, 2.82, 5.8, 177.146, 0.  , 0.  ),
           (1, 1959., 2., 2778.801, 1733.7, 310.859, 481.301, 1919.7, 29.15, 141.7, 3.08, 5.1, 177.83 , 2.34, 0.74),
           (2, 1959., 3., 2775.488, 1751.8, 289.226, 491.26 , 1916.4, 29.35, 140.5, 3.82, 5.3, 178.657, 2.74, 1.09)],
          dtype=[('index', '<i8'), ('year', '<f8'), ('quarter', '<f8'), ('realgdp', '<f8'), ('realcons', '<f8'), ('realinv', '<f8'), ('realgovt', '<f8'), ('realdpi', '<f8'), ('cpi', '<f8'), ('m1', '<f8'), ('tbilrate', '<f8'), ('unemp', '<f8'), ('pop', '<f8'), ('infl', '<f8'), ('realint', '<f8')])

In [51]:
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end')) # 단위 'Day', 분기의 마지막 날인 'end'로 표기

In [40]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [58]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')
ldata[:5]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


In [56]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [ ]:
# statistics : ggg

## 데이터 변형

In [ ]:
# 중복제거 :duplicate

In [ ]:
# 함수나 매핑

In [ ]:
# 값 치환하기

In [ ]:
# 개별화와 양자화

In [1]:
# 치환과 임의 샘플링
# 임의의 데이터에서 train, test data는 random sampling해야 한다.

In [ ]:
# 더미변수 
# str data를 숫자로 표현했을 때 발생하는 문제점 보완.

## 문자열 다루기

In [ ]:
# 정규표현식
# 자주 까먹음